# Modeling
### Alex Bass
### 8 Aug 2022  

*Description*: I have already tested out a few models in my previous notebook, but I wanted to redo some and try out the negative binomial regression model. I think it may be more straight forward to interpret and could still work with the imbalance of zeros

In [1]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import seaborn as sns
import os
import bambi as bmb

from formulae import design_matrices

In [2]:
data = pd.read_csv('/Users/alex/Library/CloudStorage/OneDrive-Personal/DS6040/Project - School Shootings/Finalized_Data/new_train.csv') #reading in final data
data.columns

Index(['STATE', 'gun_own', 'hunt_license', 'background_checks',
       'gun_permit_law', 'under40', 'Male', 'White', 'Black', 'Asian',
       'Hispanic', 'Unemployment_rate_2021', 'Median_Household_Income_2020',
       'ba_plus', 'less_than_hs', 'hs', 'some_col', 'population', 'n',
       'gun_strictness', 'rural', 'suburban', 'urban'],
      dtype='object')

In [4]:
data.shape

(3141, 24)

In [5]:
data.dropna(inplace = True) #dropping NAs

data['state_r'] = data['STATE']
data = data.replace({'state_r': {1: 0, 2: 1, 4: 2, 5: 3, 6: 4, 8: 5, 9: 6, 10: 7, 12: 8, 13: 9, 15: 10, 16: 11, 17: 12, 18: 13, 
                           19: 14, 20: 15, 21: 16, 22: 17, 23: 18, 24: 19, 25: 20, 26: 21, 27: 22, 28: 23, 29: 24, 30: 25, 
                           31: 26, 32: 27, 33: 28, 34: 29, 35: 30, 36: 31, 37: 32, 38: 33, 39: 34, 40: 35, 41: 36, 42: 37, 
                           44: 38, 45: 39, 46: 40, 47: 41, 48: 42, 49: 43, 50: 44, 51: 45, 53: 46, 54: 47, 55: 48, 56: 49 
}})

state_r = data.state_r.values
n_state = len(data.state_r.unique())

data.state_r.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [8]:
#scaling all data
columns_to_scale = ['gun_own',
       'hunt_license', 'under40', 'Male', 'White', 'Black', 'Asian',
       'Hispanic', 'Unemployment_rate_2021', 'Median_Household_Income_2020',
       'ba_plus', 'less_than_hs', 'hs', 'some_col',
       'population', 'gun_strictness']

data[columns_to_scale] = data[columns_to_scale].apply(lambda x : (x - np.mean(x))/np.std(x))

In [10]:
fml = "n ~ gun_own + hunt_license + background_checks + gun_permit_law + under40 + Male + White + Black + Asian + Hispanic + Unemployment_rate_2021 + Median_Household_Income_2020 + ba_plus + less_than_hs + hs + some_col + population + gun_strictness + urban + suburban + rural"  # full formulae formulation

dm = design_matrices(fml, data, na_action="error")

mx_ex = dm.common.as_dataframe()
mx_en = dm.response.as_dataframe()

mx_ex

,Intercept,gun_own,hunt_license,background_checks,gun_permit_law,under40,Male,White,Black,Asian,...,Median_Household_Income_2020,ba_plus,less_than_hs,hs,some_col,population,gun_strictness,urban,suburban,rural
0,1.0,0.833042,-0.186645,0.0,0.0,0.397307,-0.717680,-0.495257,0.745100,-0.136231,...,0.694312,0.589642,-0.184500,-0.346835,-0.382626,-0.145751,-1.154567,1.0,0.0,0.0
1,1.0,0.833042,-0.186645,0.0,0.0,-0.419029,-0.718403,0.187723,-0.039755,-0.172758,...,0.939418,0.960109,-0.475586,-0.917436,0.061710,0.355878,-1.154567,1.0,0.0,0.0
2,1.0,0.833042,-0.186645,0.0,0.0,0.204220,1.238609,-2.151528,2.691732,-0.374357,...,-1.276080,-1.134098,2.131007,0.245205,-0.690082,-0.239213,-1.154567,0.0,1.0,0.0
3,1.0,0.833042,-0.186645,0.0,0.0,0.386655,1.392354,-0.464478,0.825132,-0.460765,...,-0.449379,-1.161714,1.112068,1.511475,-1.235287,-0.246082,-1.154567,1.0,0.0,0.0
4,1.0,0.833042,-0.186645,0.0,0.0,0.074468,-0.388431,0.699984,-0.530321,-0.425066,...,-0.154428,-0.965131,0.790681,0.158194,0.638844,-0.139885,-1.154567,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3136,1.0,2.206241,1.079379,0.0,0.0,1.234813,0.612461,0.573391,-0.549638,-0.171511,...,0.901519,-0.143786,-0.823092,-0.224346,1.501087,-0.186291,-2.055884,0.0,1.0,0.0
3137,1.0,2.206241,1.079379,0.0,0.0,0.661028,0.720313,0.648044,-0.596664,0.035955,...,2.405455,3.348400,-1.173494,-2.484783,-1.319654,-0.242875,-2.055884,0.0,0.0,1.0
3138,1.0,2.206241,1.079379,0.0,0.0,0.884406,0.190871,0.668374,-0.595385,-0.366069,...,0.947039,-0.361401,-0.911429,0.505643,0.988468,-0.252580,-2.055884,0.0,0.0,1.0
3139,1.0,2.206241,1.079379,0.0,0.0,-0.643928,0.290855,0.620004,-0.608385,-0.255988,...,0.074132,0.124250,-0.729335,-0.699993,1.565122,-0.289808,-2.055884,0.0,0.0,1.0


In [12]:
with pm.Model() as m_neg_binomial:
    # b0 - intercept 
    mu_b0 = pm.Normal('mu_b0', mu=0., sd=5)
    sigma_b0 = pm.HalfCauchy('sigma_b0', 5)
    
    # Random intercepts as offsets
    a_offset = pm.Normal('a_offset', mu=0, sd=1, shape=n_state)
    b0 = pm.Deterministic("a", mu_b0 + a_offset * sigma_b0)
    
    #Betas
    b = pm.Normal("slopes", mu=0, sigma=5, shape=21)
    
    #Alpha
    alpha = pm.Exponential("alpha", 0.5)
    
    λ = pm.math.exp(
        b0[state_r]
        + b[0] * mx_ex["gun_own"].values
        + b[1] * mx_ex["background_checks"].values
        + b[2] * mx_ex["gun_permit_law"].values
        + b[3] * mx_ex["gun_strictness"].values
        + b[4] * mx_ex["hunt_license"].values
        + b[5] * mx_ex["under40"].values
        + b[6] * mx_ex["Male"].values
        + b[7] * mx_ex["White"].values
        + b[8] * mx_ex["Black"].values
        + b[9] * mx_ex["Asian"].values
        + b[10] * mx_ex["Hispanic"].values
        + b[11] * mx_ex["Unemployment_rate_2021"].values
        + b[12] * mx_ex["Median_Household_Income_2020"].values
        + b[13] * mx_ex["ba_plus"].values
        + b[14] * mx_ex["less_than_hs"].values
        + b[15] * mx_ex["hs"].values
        + b[16] * mx_ex["some_col"].values
        + b[17] * mx_ex["urban"].values
        + b[18] * mx_ex["suburban"].values
        + b[19] * mx_ex["rural"].values
        + b[20] * mx_ex["population"].values
    )

    y = pm.NegativeBinomial("y", mu=λ, alpha=alpha, observed=mx_en)

    trace_1 = pm.sample()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, slopes, a_offset, sigma_b0, mu_b0]


In [ ]:
az.plot_trace(trace_1, compact=False)

In [ ]:
az.summary(np.exp(trace_1.posterior), kind="stats", var_names=["intercept", "slopes"])